Granulometric Filter
=================
An implementration of the Granulometric Filter in Python. See https://imagej.net/Granulometric_Filtering.

In [ ]:
import numpy as np
import numpy.fft as fft
import scipy.signal as sgnl
import scipy.ndimage as ndi
import skimage.io as io
import skimage.filters as fltrs
import skimage.feature as ftr
import matplotlib.pylab as plt
from skimage.util import random_noise
from zerocross import zerocross
plt.gray()

In [ ]:
im = io.imread('cameraman.tif') / 255

1) As a first step, the original color images must be converted to grayscale. 

2) Afterwards, a smoothing filter attenuates the  intensity of particle’s contour, this filter can discriminate very small changes in the shape of the particle. Smoothing convolution filter is an averaging filter whose kernel uses the following model:
$$
\begin{bmatrix}
a & d & c \\
b & 0 & b \\
c & d & a
\end{bmatrix}
$$
where $a$, $b$, $c$, and $d$ are positive integers.

3) Segments the  image  using  the clustering method. The  method is to separate the histograms of intensities in a discrete number of phases. Clustering sorts the histogram of the image within a discrete number of classes corresponding to the number of phases perceived in an image. The gray values are determined, and a barycenter is determined for each class. This process repeats until it obtains a value that represents the center of mass for each phase or class. The threshold value is the pixel value k for which the following condition is true:  ݇ $$k=\frac{\mu_1-\mu_2}{2}$$ where $\mu_1$ is the mean of all pixel values that lie between 0 and k, and $\mu_2$ is the mean of all the pixel values that lie between k + 1 and 255.

4) A low pass filter removes small particles. These are eliminated because they do not span enough pixels. The low pass filter removes small particles according to their widths as specified by a parameter called filter size. For a given filter size N, the lowpass filter eliminates particles whose widths are less than or equal to (N – 1) pixels. These particles disappear after (N – 1)/2 erosions.

5) Particles that are located on the edge of the image are removed. The elimination of these particles avoids an erroneous measurement because they are not completely visible.

6) Some particles have different phases, therefore, at the time of segmentation there are some holes on the particle. A function is used to fill these holes.

In [ ]:
def __smooth_filter(fltr):
    # TODO: Test default filter to find best or good enough values
    default_fltr = (1, 1, 1, 1)
    # Make sure smooth filter values are positive integers
    for i in range(4):
        if fltr[i] < 1 and fltr[i] != int(fltr[i]):
            fltr[i] = default_fltr[i]
    a, b, c, d = fltr
    smooth_filter = np.array([[a, d, c],
                              [b, 0, b],
                              [c, d, a]])
    return ndi.convolve(im, smooth_filter)

def granulometric_filter(im, fltr=[1,1,1,1], remove_edge=True):
    '''im: a grayscale image'''
    # TODO: Apply smoothing filter
    im_smooth = __smooth_filter(im, fltr)

    # TODO: Segment image using the clustering method

    # TODO: Apply low pass filter to remove small particles

    # TODO: Remove particles on the edge of the image
    # Doesn't necessarily have to be done if the edge particles aren't obscured; can make optional

    # TODO: Fill in particles with holes in them

    return im_smooth